# Context

In [1]:
# Assume you are working as a data analyst in a data science team, your task is to: 
# 1.examine the dataset 
# 2.cleaning and preprocessing 
# 3.save it to an external .csv file
# and then pass it to the data scientists to build a Credit Risk Model that measures the probability of default

## Importing Packages

In [2]:
import numpy as np

In [3]:
np.set_printoptions(suppress = True, linewidth = 100, precision = 2) 
#suppress: if True, always print floating point numbers using fixed point notation, in which case numbers equal to zero in the current precision will print as zero
#precision: number of digits of precision for floating point output

## Import the Data

In [4]:
#raw_data_np = np.loadtxt('loan-data.csv', delimiter = ';')
## np.loadtxt() will return and error message if the data is incomplete
## fail to generate float from string

#raw_data_np

In [5]:
raw_data_np = np.genfromtxt('loan-data.csv', delimiter = ';', encoding='unicode_escape', skip_header =1, autostrip = True)
raw_data_np

array([[48010226.  ,         nan,    35000.  , ...,         nan,         nan,     9452.96],
       [57693261.  ,         nan,    30000.  , ...,         nan,         nan,     4679.7 ],
       [59432726.  ,         nan,    15000.  , ...,         nan,         nan,     1969.83],
       ...,
       [50415990.  ,         nan,    10000.  , ...,         nan,         nan,     2185.64],
       [46154151.  ,         nan,         nan, ...,         nan,         nan,     3199.4 ],
       [66055249.  ,         nan,    10000.  , ...,         nan,         nan,      301.9 ]])

## Checking for Incomplete Data

In [6]:
np.isnan(raw_data_np).sum() #88005 missing values

88005

In [7]:
temporary_fill = np.nanmax(raw_data_np) +1  #a filler for all the missing entries of the dataset
temporary_mean = np.nanmean(raw_data_np, axis = 0)  #hold the means for every column

#from the warning, there must exist entire columns full of only NaNs 
#a column of only strings is automatically filled with NaNs (and considered 'empty', but != to 'empty') 

<ipython-input-7-8d9450ebf1c9>:2: RuntimeWarning: Mean of empty slice
  temporary_mean = np.nanmean(raw_data_np, axis = 0)  #hold the means for every column


In [8]:
temporary_mean
#some coluns only store strings, contains no numbers

array([54015809.19,         nan,    15273.46,         nan,    15311.04,         nan,       16.62,
            440.92,         nan,         nan,         nan,         nan,         nan,     3143.85])

In [9]:
temporary_stats = np.array([np.nanmin(raw_data_np, axis=0), temporary_mean, np.nanmax(raw_data_np, axis=0)])
temporary_stats

<ipython-input-9-16f840aeb93e>:1: RuntimeWarning: All-NaN slice encountered
  temporary_stats = np.array([np.nanmin(raw_data_np, axis=0), temporary_mean, np.nanmax(raw_data_np, axis=0)])


array([[  373332.  ,         nan,     1000.  ,         nan,     1000.  ,         nan,        6.  ,
              31.42,         nan,         nan,         nan,         nan,         nan,        0.  ],
       [54015809.19,         nan,    15273.46,         nan,    15311.04,         nan,       16.62,
             440.92,         nan,         nan,         nan,         nan,         nan,     3143.85],
       [68616519.  ,         nan,    35000.  ,         nan,    35000.  ,         nan,       28.99,
            1372.97,         nan,         nan,         nan,         nan,         nan,    41913.62]])

# Splitting the Dataset

## Splitting the Columns

In [10]:
np.isnan(temporary_mean)

array([False,  True, False,  True, False,  True, False, False,  True,  True,  True,  True,  True,
       False])

In [11]:
columns_strings = np.argwhere(np.isnan(temporary_mean))   #argwhere takes condition, returns the column index
columns_strings

array([[ 1],
       [ 3],
       [ 5],
       [ 8],
       [ 9],
       [10],
       [11],
       [12]], dtype=int64)

In [12]:
columns_strings = np.argwhere(np.isnan(temporary_mean)).squeeze()  #putting all elements into 1-D array
columns_strings

array([ 1,  3,  5,  8,  9, 10, 11, 12], dtype=int64)

In [13]:
columns_numeric = np.argwhere(np.isnan(temporary_mean)== False).squeeze()  
columns_numeric

array([ 0,  2,  4,  6,  7, 13], dtype=int64)

## Re-importing the Dataset

In [14]:
strings_data = np.genfromtxt('loan-data.csv', 
                             delimiter = ';', 
                             encoding='unicode_escape', 
                             skip_header =1, 
                             autostrip = True, 
                             dtype = np.str,
                             usecols = columns_strings) #usecols takes specific columns
strings_data

<ipython-input-14-64ecfe8548ad>:6: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.str,


array([['May-15', 'Current', '36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226', 'CA'],
       ['', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261', 'NY'],
       ['Sep-15', 'Current', '36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726', 'PA'],
       ...,
       ['Jun-15', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=50415990', 'CA'],
       ['Apr-15', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=46154151', 'OH'],
       ['Dec-15', 'Current', '36 months', ..., '',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=66055249', 'IL']],
      dtype='<U69')

In [15]:
numeric_data = np.genfromtxt('loan-data.csv', 
                             delimiter = ';', 
                             encoding='unicode_escape', 
                             skip_header =1, 
                             autostrip = True, 
                             usecols = columns_numeric) #usecols takes specific columns
numeric_data

array([[48010226.  ,    35000.  ,    35000.  ,       13.33,     1184.86,     9452.96],
       [57693261.  ,    30000.  ,    30000.  ,         nan,      938.57,     4679.7 ],
       [59432726.  ,    15000.  ,    15000.  ,         nan,      494.86,     1969.83],
       ...,
       [50415990.  ,    10000.  ,    10000.  ,         nan,         nan,     2185.64],
       [46154151.  ,         nan,    10000.  ,       16.55,      354.3 ,     3199.4 ],
       [66055249.  ,    10000.  ,    10000.  ,         nan,      309.97,      301.9 ]])

In [16]:
numeric_data = np.genfromtxt('loan-data.csv', 
                             delimiter = ';', 
                             encoding='unicode_escape', 
                             skip_header =1, 
                             autostrip = True, 
                             filling_values = temporary_fill, #filling nan
                             usecols = columns_numeric) #usecols takes specific columns
numeric_data

array([[48010226.  ,    35000.  ,    35000.  ,       13.33,     1184.86,     9452.96],
       [57693261.  ,    30000.  ,    30000.  , 68616520.  ,      938.57,     4679.7 ],
       [59432726.  ,    15000.  ,    15000.  , 68616520.  ,      494.86,     1969.83],
       ...,
       [50415990.  ,    10000.  ,    10000.  , 68616520.  , 68616520.  ,     2185.64],
       [46154151.  , 68616520.  ,    10000.  ,       16.55,      354.3 ,     3199.4 ],
       [66055249.  ,    10000.  ,    10000.  , 68616520.  ,      309.97,      301.9 ]])

In [17]:
header_full = np.genfromtxt('loan-data.csv', 
                             delimiter = ';', 
                             encoding='unicode_escape',  
                             skip_footer = raw_data_np.shape[0], #number of rows of raw data
                             autostrip = True, 
                             dtype = np.str,
                             ) 
header_full

<ipython-input-17-ecc6d9e84a98>:6: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.str,


array(['id', 'issue_d', 'loan_amnt', 'loan_status', 'funded_amnt', 'term', 'int_rate',
       'installment', 'grade', 'sub_grade', 'verification_status', 'url', 'addr_state',
       'total_pymnt'], dtype='<U19')

In [18]:
header_strings = header_full[columns_strings]
header_strings

array(['issue_d', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

In [19]:
header_numeric = header_full[columns_numeric]
header_numeric

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt'], dtype='<U19')

# Creating Checkpoints: 

In [20]:
def checkpoint(file_name, checkpoint_header, checkpoint_data):
    np.savez(file_name, header=checkpoint_header, data = checkpoint_data)
    checkpoint_variable = np.load(file_name + '.npz')
    return(checkpoint_variable)

In [21]:
checkpoint_test = checkpoint("checkpoint-test", header_strings, strings_data)

In [22]:
checkpoint_test['header']

array(['issue_d', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

In [23]:
checkpoint_test['data']

array([['May-15', 'Current', '36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226', 'CA'],
       ['', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261', 'NY'],
       ['Sep-15', 'Current', '36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726', 'PA'],
       ...,
       ['Jun-15', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=50415990', 'CA'],
       ['Apr-15', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=46154151', 'OH'],
       ['Dec-15', 'Current', '36 months', ..., '',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=66055249', 'IL']],
      dtype='<U69')

# Manipulating String Columns

In [24]:
header_strings

array(['issue_d', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

In [25]:
header_strings[0] = 'issue_data'

In [26]:
header_strings

array(['issue_data', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

In [27]:
strings_data  

array([['May-15', 'Current', '36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226', 'CA'],
       ['', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261', 'NY'],
       ['Sep-15', 'Current', '36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726', 'PA'],
       ...,
       ['Jun-15', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=50415990', 'CA'],
       ['Apr-15', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=46154151', 'OH'],
       ['Dec-15', 'Current', '36 months', ..., '',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=66055249', 'IL']],
      dtype='<U69')

## Issue Date

In [28]:
strings_data[:,0]

array(['May-15', '', 'Sep-15', ..., 'Jun-15', 'Apr-15', 'Dec-15'], dtype='<U69')

In [29]:
np.unique(strings_data[:,0]) #all the data are from 2015, and thus we can strip '-15'

array(['', 'Apr-15', 'Aug-15', 'Dec-15', 'Feb-15', 'Jan-15', 'Jul-15', 'Jun-15', 'Mar-15',
       'May-15', 'Nov-15', 'Oct-15', 'Sep-15'], dtype='<U69')

In [30]:
strings_data[:,0] = np.char.strip(strings_data[:,0], '-15')

In [31]:
strings_data[:,0]

array(['May', '', 'Sep', ..., 'Jun', 'Apr', 'Dec'], dtype='<U69')

In [32]:
np.unique(strings_data[:,0])

array(['', 'Apr', 'Aug', 'Dec', 'Feb', 'Jan', 'Jul', 'Jun', 'Mar', 'May', 'Nov', 'Oct', 'Sep'],
      dtype='<U69')

In [33]:
months = np.array(['','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'])

In [34]:
for i in range (13): 
    strings_data[:,0] = np.where(strings_data[:,0] == months[i], #np.where, iterate over string data. IMPORTANT!
                                 i, 
                                 strings_data[:,0])
# what is i in this case, why would it be a str??

In [35]:
strings_data[:,0]

array(['5', '0', '9', ..., '6', '4', '12'], dtype='<U69')

In [36]:
np.unique(strings_data[:,0])

array(['0', '1', '10', '11', '12', '2', '3', '4', '5', '6', '7', '8', '9'], dtype='<U69')

## Loan Status

In [37]:
header_strings

array(['issue_data', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

In [38]:
header_strings[1]

'loan_status'

In [39]:
strings_data[:,1]

array(['Current', 'Current', 'Current', ..., 'Current', 'Current', 'Current'], dtype='<U69')

In [40]:
np.unique(strings_data[:,1])

array(['', 'Charged Off', 'Current', 'Default', 'Fully Paid', 'In Grace Period', 'Issued',
       'Late (16-30 days)', 'Late (31-120 days)'], dtype='<U69')

In [41]:
np.unique(strings_data[:,1]).size

9

In [42]:
status_bad = np.array(['','Charged Off','Default','Late (31-120 days)']) 
#these status are treated as bad --> 0 (1 for good)

In [43]:
strings_data[:,1] = np.where(np.isin(strings_data[:,1],status_bad),0,1)
# if the status is in status_bad assign 0, otherwise assign 1

In [44]:
np.unique(strings_data[:,1])

array(['0', '1'], dtype='<U69')

## Term

In [45]:
strings_data[:,2]

array(['36 months', '36 months', '36 months', ..., '36 months', '36 months', '36 months'],
      dtype='<U69')

In [46]:
np.unique(strings_data[:,2])

array(['', '36 months', '60 months'], dtype='<U69')

In [47]:
strings_data[:,2] = np.char.strip(strings_data[:,2], ' months') 

In [48]:
strings_data[:,2]

array(['36', '36', '36', ..., '36', '36', '36'], dtype='<U69')

In [49]:
np.unique(strings_data[:,2])

array(['', '36', '60'], dtype='<U69')

In [50]:
header_strings[2] = "term_month"

In [51]:
strings_data[:,2] = np.where(strings_data[:,2] == '', '60', strings_data[:,2])

In [52]:
np.unique(strings_data[:,2])

array(['36', '60'], dtype='<U69')

## Grade and subgrade

In [53]:
header_strings

array(['issue_data', 'loan_status', 'term_month', 'grade', 'sub_grade', 'verification_status',
       'url', 'addr_state'], dtype='<U19')

In [54]:
strings_data[:,3]

array(['C', 'A', 'B', ..., 'A', 'D', 'A'], dtype='<U69')

In [55]:
np.unique(strings_data[:,3])

array(['', 'A', 'B', 'C', 'D', 'E', 'F', 'G'], dtype='<U69')

In [56]:
strings_data[:,4]

array(['C3', 'A5', 'B5', ..., 'A5', 'D2', 'A4'], dtype='<U69')

In [57]:
np.unique(strings_data[:,4])

array(['', 'A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4',
       'C5', 'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4',
       'F5', 'G1', 'G2', 'G3', 'G4', 'G5'], dtype='<U69')

### Filling Sub Grade

In [58]:
empty_location = strings_data[1,:]

In [59]:
for i in np.unique(strings_data[:,3])[1:]:
    # use [1:] to skip the first one, as it is just an empty space
    strings_data[:,4] = np.where((strings_data[:,4]=='') & (strings_data[:,3] == i), 
                                        i + '5', #sub with the worst subgrade in that missing value
                                        strings_data[:,4])


In [60]:
np.unique(strings_data[:,4])
#we still have missing data

array(['', 'A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4',
       'C5', 'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4',
       'F5', 'G1', 'G2', 'G3', 'G4', 'G5'], dtype='<U69')

In [61]:
np.unique(strings_data[:,4], return_counts = True)   #return_counts how many times each subgrade occured

(array(['', 'A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4',
        'C5', 'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4',
        'F5', 'G1', 'G2', 'G3', 'G4', 'G5'], dtype='<U69'),
 array([  9, 285, 278, 239, 323, 592, 509, 517, 530, 553, 633, 629, 567, 586, 564, 577, 391, 267,
        250, 255, 288, 235, 162, 171, 139, 160,  94,  52,  34,  43,  24,  19,  10,   3,   7,   5],
       dtype=int64))

In [62]:
strings_data[:,4] = np.where((strings_data[:,4]==''), 
                                        'H1',
                                        strings_data[:,4])
#sub the subgrade with a missing value that does have a grade to begin with, with a new grade H1

In [63]:
np.unique(strings_data[:,4])

array(['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4', 'C5',
       'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4', 'F5',
       'G1', 'G2', 'G3', 'G4', 'G5', 'H1'], dtype='<U69')

### Removing Grade

In [64]:
#all the information in grade is now carried by subgrade, so we don't need it anymore 
strings_data = np.delete(strings_data, 3, axis = 1) #remove 4th value along second axis, aka 4th column

In [65]:
strings_data[:,3]

array(['C3', 'A5', 'B5', ..., 'A5', 'D2', 'A4'], dtype='<U69')

In [66]:
header_strings

array(['issue_data', 'loan_status', 'term_month', 'grade', 'sub_grade', 'verification_status',
       'url', 'addr_state'], dtype='<U19')

In [67]:
header_strings = np.delete(header_strings,3)

In [68]:
header_strings[3]

'sub_grade'

### Converting Sub Grade

In [69]:
strings_data[:,3]

array(['C3', 'A5', 'B5', ..., 'A5', 'D2', 'A4'], dtype='<U69')

In [70]:
#np.unique(strings_data[:,3]).size

In [71]:
#np.unique(strings_data[:,3]).shape[0]

In [72]:
#we dont want to manually use the np.where() function multiple times, so we will use dictionary

#dictionary: keys, values 
#keys
keys = list(np.unique(strings_data[:,3]))
#values
values = list(range(1,np.unique(strings_data[:,3]).size+1))
dict_sub_grade = dict(zip(keys,values)) #dict and zip!!!
dict_sub_grade

{'A1': 1,
 'A2': 2,
 'A3': 3,
 'A4': 4,
 'A5': 5,
 'B1': 6,
 'B2': 7,
 'B3': 8,
 'B4': 9,
 'B5': 10,
 'C1': 11,
 'C2': 12,
 'C3': 13,
 'C4': 14,
 'C5': 15,
 'D1': 16,
 'D2': 17,
 'D3': 18,
 'D4': 19,
 'D5': 20,
 'E1': 21,
 'E2': 22,
 'E3': 23,
 'E4': 24,
 'E5': 25,
 'F1': 26,
 'F2': 27,
 'F3': 28,
 'F4': 29,
 'F5': 30,
 'G1': 31,
 'G2': 32,
 'G3': 33,
 'G4': 34,
 'G5': 35,
 'H1': 36}

In [73]:
for i in np.unique(strings_data[:,3]):
    strings_data[:,3] = np.where(strings_data[:,3] == i,
                                dict_sub_grade[i],
                                strings_data[:,3])

In [74]:
strings_data[:,3]

array(['13', '5', '10', ..., '5', '17', '4'], dtype='<U69')

In [75]:
np.unique(strings_data[:,3])

array(['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22',
       '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36',
       '4', '5', '6', '7', '8', '9'], dtype='<U69')

In [76]:
np.unique(np.unique(strings_data[:,3]))

array(['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22',
       '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36',
       '4', '5', '6', '7', '8', '9'], dtype='<U69')

### Verification status

In [77]:
header_strings

array(['issue_data', 'loan_status', 'term_month', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

In [78]:
strings_data[:,4]

array(['Verified', 'Source Verified', 'Verified', ..., 'Source Verified', 'Source Verified', ''],
      dtype='<U69')

In [79]:
np.unique(strings_data[:,4])

array(['', 'Not Verified', 'Source Verified', 'Verified'], dtype='<U69')

In [80]:
strings_data[:,4] = np.where((strings_data[:,4]=='')|(strings_data[:,4]=='Not Verified'),0,1)

In [81]:
strings_data[:,4]

array(['1', '1', '1', ..., '1', '1', '0'], dtype='<U69')

In [82]:
np.unique(strings_data[:,4])

array(['0', '1'], dtype='<U69')

### URL

In [83]:
header_strings

array(['issue_data', 'loan_status', 'term_month', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

In [84]:
strings_data[:,5]

array(['https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226',
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261',
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726', ...,
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=50415990',
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=46154151',
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=66055249'], dtype='<U69')

In [85]:
strings_data[:,5] = np.chararray.strip(strings_data[:,5], 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=')

In [86]:
strings_data[:,5]

array(['48010226', '57693261', '59432726', ..., '50415990', '46154151', '66055249'], dtype='<U69')

In [87]:
strings_data[:,5].astype(dtype=np.int32)

array([48010226, 57693261, 59432726, ..., 50415990, 46154151, 66055249])

In [88]:
header_full[0]

'id'

In [89]:
numeric_data[:,0].astype(dtype = np.int32)

array([48010226, 57693261, 59432726, ..., 50415990, 46154151, 66055249])

In [90]:
#the number we got from URL seems identical to the ID column, we will check below
np.array_equal(strings_data[:,5].astype(dtype=np.int32),numeric_data[:,0].astype(dtype = np.int32))


True

In [91]:
#returned True confirm that the URL column doesn't hold any additional information we can't extract from the ID column
#and thus, we will delete this URL column
strings_data = np.delete(strings_data, 5, axis = 1)

In [92]:
header_strings = np.delete(header_strings,5)

In [93]:
header_strings

array(['issue_data', 'loan_status', 'term_month', 'sub_grade', 'verification_status', 'addr_state'],
      dtype='<U19')

In [94]:
strings_data

array([['5', '1', '36', '13', '1', 'CA'],
       ['0', '1', '36', '5', '1', 'NY'],
       ['9', '1', '36', '10', '1', 'PA'],
       ...,
       ['6', '1', '36', '5', '1', 'CA'],
       ['4', '1', '36', '17', '1', 'OH'],
       ['12', '1', '36', '4', '0', 'IL']], dtype='<U69')

### State Address

In [95]:
header_strings[5]

'addr_state'

In [96]:
strings_data[:,5]

array(['CA', 'NY', 'PA', ..., 'CA', 'OH', 'IL'], dtype='<U69')

In [97]:
np.unique(strings_data[:,5])

array(['', 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IL', 'IN',
       'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH',
       'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA',
       'VT', 'WA', 'WI', 'WV', 'WY'], dtype='<U69')

In [98]:
np.unique(strings_data[:,5]).size

50

In [99]:
#there are 50 unique values with 1 empty value, we can find out it's IA
#we suspect it was purposefully left as a baseline benchmark
#when doing reseasrch or analysis on a variable with many categories
#it is normal to pick one as a benchmark and include dummy variables for the rest

In [100]:
states_names, states_count = np.unique(strings_data[:,5], return_counts = True) #states_names and states_count are assigned to corresponding arrays

In [101]:
states_count_sorted = np.argsort(-states_count) #IMPORTANT argsort returns an array of indices of the same shape as the array that index data along the given axis in sorted order
states_count_sorted

array([ 5, 33, 42, 10,  0, 13, 30, 11, 37, 34, 21, 26, 44, 19,  4, 46, 18,  6, 23, 22, 14, 47,  7,
       41, 32,  2, 17, 36, 39, 16, 15, 35, 43,  3, 24, 29, 31, 48, 12, 38, 25,  9,  8, 49,  1, 28,
       40, 45, 27, 20], dtype=int64)

In [102]:
states_names[states_count_sorted],states_count[states_count_sorted]

(array(['CA', 'NY', 'TX', 'FL', '', 'IL', 'NJ', 'GA', 'PA', 'OH', 'MI', 'NC', 'VA', 'MD', 'AZ',
        'WA', 'MA', 'CO', 'MO', 'MN', 'IN', 'WI', 'CT', 'TN', 'NV', 'AL', 'LA', 'OR', 'SC', 'KY',
        'KS', 'OK', 'UT', 'AR', 'MS', 'NH', 'NM', 'WV', 'HI', 'RI', 'MT', 'DE', 'DC', 'WY', 'AK',
        'NE', 'SD', 'VT', 'ND', 'ME'], dtype='<U69'),
 array([1336,  777,  758,  690,  500,  389,  341,  321,  320,  312,  267,  261,  242,  222,  220,
         216,  210,  201,  160,  156,  152,  148,  143,  143,  130,  119,  116,  108,  107,   84,
          84,   83,   74,   74,   61,   58,   57,   49,   44,   40,   28,   27,   27,   27,   26,
          25,   24,   17,   16,   10], dtype=int64))

In [103]:
#we have very little data for too many states to examine each one individually
#if we assign a unique value to each state, thsi will allow outliers to have a big influence on the coefficients
#we need to group the states with common characteristics, can be geographic location in this case

In [104]:
#account can't be contributed to a specific state wont be included in a region, to avoid biasing coefficient
strings_data[:,5] = np.where(strings_data[:,5] =='',0, strings_data[:,5])

In [105]:
np.unique(strings_data[:,5])

array(['0', 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IL', 'IN',
       'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH',
       'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA',
       'VT', 'WA', 'WI', 'WV', 'WY'], dtype='<U69')

In [106]:
states_west = np.array(['WA','OR','CA','NV','ID','MT','WY','UT','CO','AZ','NM','HI','AK'])
states_south = np.array(['TX','OK','AR','LA','MS','AL','TN','KY','FL','GA','SC','NC','VA','WV','MD','DE','DC'])
states_midwest = np.array(['ND','SD','NE','KS','MN','IA','MO','WI','IL','IN','MI','OH'])
states_east = np.array(['PA','NY','NJ','CT','MA','VT','NH','ME','RI'])

In [107]:
strings_data[:,5] = np.where(np.isin(strings_data[:,5],states_west),1,strings_data[:,5])
strings_data[:,5] = np.where(np.isin(strings_data[:,5],states_south),2,strings_data[:,5])
strings_data[:,5] = np.where(np.isin(strings_data[:,5],states_midwest),3,strings_data[:,5])
strings_data[:,5] = np.where(np.isin(strings_data[:,5],states_east),4,strings_data[:,5])

In [108]:
np.unique(strings_data[:,5])

array(['0', '1', '2', '3', '4'], dtype='<U69')

In [109]:
np.unique(strings_data[:,5]).size

5

In [110]:
strings_data
#we have converted whatever string data we had into numeric values stored as text

array([['5', '1', '36', '13', '1', '1'],
       ['0', '1', '36', '5', '1', '4'],
       ['9', '1', '36', '10', '1', '4'],
       ...,
       ['6', '1', '36', '5', '1', '1'],
       ['4', '1', '36', '17', '1', '3'],
       ['12', '1', '36', '4', '0', '3']], dtype='<U69')

### Convertig to Numbers

In [111]:
strings_data.astype(np.int) # .astype to convert str to int

<ipython-input-111-ba64bcdac52b>:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  strings_data.astype(np.int) # .astype to convert str to int


array([[ 5,  1, 36, 13,  1,  1],
       [ 0,  1, 36,  5,  1,  4],
       [ 9,  1, 36, 10,  1,  4],
       ...,
       [ 6,  1, 36,  5,  1,  1],
       [ 4,  1, 36, 17,  1,  3],
       [12,  1, 36,  4,  0,  3]])

In [112]:
strings_data = strings_data.astype(np.int)

<ipython-input-112-9fa3f0a75aa1>:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  strings_data = strings_data.astype(np.int)


### Checkpioint

In [113]:
#def checkpoint (file_name, checkpoint_header, checpoint_data): 
    #np.savez(file_name, header = checkpoint_header, data = checpoint_data)
    #return(file_name)

In [114]:
checkpoint_strings = checkpoint('checkpoint-strings', header_strings, strings_data)

In [115]:
checkpoint_strings['header']

array(['issue_data', 'loan_status', 'term_month', 'sub_grade', 'verification_status', 'addr_state'],
      dtype='<U19')

In [116]:
string_data_with_deader = np.load('checkpoint-strings.npz')

In [117]:
string_data_with_deader['header']

array(['issue_data', 'loan_status', 'term_month', 'sub_grade', 'verification_status', 'addr_state'],
      dtype='<U19')

## Manipulating Numeric Data

In [118]:
header_numeric

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt'], dtype='<U19')

In [119]:
numeric_data

array([[48010226.  ,    35000.  ,    35000.  ,       13.33,     1184.86,     9452.96],
       [57693261.  ,    30000.  ,    30000.  , 68616520.  ,      938.57,     4679.7 ],
       [59432726.  ,    15000.  ,    15000.  , 68616520.  ,      494.86,     1969.83],
       ...,
       [50415990.  ,    10000.  ,    10000.  , 68616520.  , 68616520.  ,     2185.64],
       [46154151.  , 68616520.  ,    10000.  ,       16.55,      354.3 ,     3199.4 ],
       [66055249.  ,    10000.  ,    10000.  , 68616520.  ,      309.97,      301.9 ]])

In [120]:
np.isnan(numeric_data).sum()

0

In [121]:
temporary_fill

68616520.0

In [122]:
#check if temporary_fill is in the id column
np.isin(numeric_data[:,0],temporary_fill)

array([False, False, False, ..., False, False, False])

In [123]:
np.isin(numeric_data[:,0],temporary_fill).sum() #sum()returns the total number of True's 

0

In [124]:
header_numeric

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt'], dtype='<U19')

### Temporary Stats

In [125]:
temporary_stats

array([[  373332.  ,         nan,     1000.  ,         nan,     1000.  ,         nan,        6.  ,
              31.42,         nan,         nan,         nan,         nan,         nan,        0.  ],
       [54015809.19,         nan,    15273.46,         nan,    15311.04,         nan,       16.62,
             440.92,         nan,         nan,         nan,         nan,         nan,     3143.85],
       [68616519.  ,         nan,    35000.  ,         nan,    35000.  ,         nan,       28.99,
            1372.97,         nan,         nan,         nan,         nan,         nan,    41913.62]])

In [126]:
#temporary stats for numeric data
temporary_stats[:, columns_numeric]

array([[  373332.  ,     1000.  ,     1000.  ,        6.  ,       31.42,        0.  ],
       [54015809.19,    15273.46,    15311.04,       16.62,      440.92,     3143.85],
       [68616519.  ,    35000.  ,    35000.  ,       28.99,     1372.97,    41913.62]])

### Funded Amount

In [127]:
#missing value will be set to minimun
numeric_data[:,2]

array([35000., 30000., 15000., ..., 10000., 10000., 10000.])

In [128]:
numeric_data[:,2] = np.where(numeric_data[:,2]==temporary_fill, 
                             temporary_stats[0,columns_numeric[2]], 
                             numeric_data[:,2])
numeric_data[:,2]


array([35000., 30000., 15000., ..., 10000., 10000., 10000.])

### loaned Amount, Interest Rate, Total Payment, Installment

In [129]:
#missing value in these columns will be set to max in its corresponding Temporary_stats

In [130]:
header_numeric

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt'], dtype='<U19')

In [131]:
for i in [1,3,4,5]: 
    numeric_data[:,i] = np.where(numeric_data[:,i]==temporary_fill, 
                             temporary_stats[2,columns_numeric[i]], 
                             numeric_data[:,i])

In [132]:
numeric_data

array([[48010226.  ,    35000.  ,    35000.  ,       13.33,     1184.86,     9452.96],
       [57693261.  ,    30000.  ,    30000.  ,       28.99,      938.57,     4679.7 ],
       [59432726.  ,    15000.  ,    15000.  ,       28.99,      494.86,     1969.83],
       ...,
       [50415990.  ,    10000.  ,    10000.  ,       28.99,     1372.97,     2185.64],
       [46154151.  ,    35000.  ,    10000.  ,       16.55,      354.3 ,     3199.4 ],
       [66055249.  ,    10000.  ,    10000.  ,       28.99,      309.97,      301.9 ]])

## Currency Change

### The Exchange Rate

In [133]:
import os
EUR_USD = np.genfromtxt(r'C:\Users\yzsty\Downloads\9_1_eur-usd-exchange-rate-2015/EUR-USD.CSV', delimiter = ',', autostrip = True)
EUR_USD
#This file contains the interest rate for each month in 2015

array([[ nan,  nan,  nan,  nan,  nan],
       [1.21, 1.21, 1.11, 1.13, 0.  ],
       [1.13, 1.15, 1.12, 1.12, 0.  ],
       [1.12, 1.12, 1.05, 1.08, 0.  ],
       [1.07, 1.12, 1.05, 1.11, 0.  ],
       [1.12, 1.15, 1.08, 1.1 , 0.  ],
       [1.1 , 1.14, 1.09, 1.12, 0.  ],
       [1.11, 1.12, 1.08, 1.09, 0.  ],
       [1.1 , 1.17, 1.09, 1.13, 0.  ],
       [1.12, 1.15, 1.11, 1.13, 0.  ],
       [1.12, 1.15, 1.09, 1.1 , 0.  ],
       [1.1 , 1.11, 1.06, 1.06, 0.  ],
       [1.06, 1.11, 1.05, 1.09, 0.  ]])

In [134]:
EUR_USD = np.genfromtxt(r'C:\Users\yzsty\Downloads\9_1_eur-usd-exchange-rate-2015/EUR-USD.CSV', delimiter = ',', autostrip = True, dtype=str)
EUR_USD

array([['Open', 'High', 'Low', 'Close', 'Volume'],
       ['1.2098628282546997', '1.2098628282546997', '1.11055588722229', '1.1287955045700073', '0'],
       ['1.1287955045700073', '1.1484194993972778', '1.117680549621582', '1.1205360889434814',
        '0'],
       ['1.119795799255371', '1.1240400075912476', '1.0460032224655151', '1.0830246210098267',
        '0'],
       ['1.0741022825241089', '1.1247594356536865', '1.0521597862243652', '1.1114321947097778',
        '0'],
       ['1.1215037107467651', '1.145304799079895', '1.0821995735168457', '1.0960345268249512',
        '0'],
       ['1.095902442932129', '1.1428401470184326', '1.0888904333114624', '1.122296690940857', '0'],
       ['1.1134989261627197', '1.1219995021820068', '1.081270456314087', '1.0939244031906128',
        '0'],
       ['1.0969001054763794', '1.1705996990203857', '1.0850305557250977', '1.1340054273605347',
        '0'],
       ['1.1225990056991577', '1.1460003852844238', '1.1089695692062378', '1.1255937814712524

In [135]:
EUR_USD = np.genfromtxt(r'C:\Users\yzsty\Downloads\9_1_eur-usd-exchange-rate-2015/EUR-USD.CSV', delimiter = ',', autostrip = True, skip_header=1,
                       usecols=3)
EUR_USD
#We will use the closing rate for the calculation

array([1.13, 1.12, 1.08, 1.11, 1.1 , 1.12, 1.09, 1.13, 1.13, 1.1 , 1.06, 1.09])

In [136]:
strings_data[:,0]
#The first column is issue month, we will use this column to map the corresponding exchange rate 

array([ 5,  0,  9, ...,  6,  4, 12])

In [137]:
exchange_rate = strings_data[:,0]
exchange_rate

array([ 5,  0,  9, ...,  6,  4, 12])

In [138]:
EUR_USD.mean()

1.1056866943836212

In [139]:
for i in range (1,13):
    exchange_rate = np.where(exchange_rate == i, 
                             EUR_USD[i-1], 
                             exchange_rate)

exchange_rate = np.where(exchange_rate == 0, 1.10, exchange_rate)    
    
exchange_rate

array([1.1 , 1.1 , 1.13, ..., 1.12, 1.11, 1.09])

In [140]:
exchange_rate.shape

(10000,)

In [141]:
numeric_data.shape

(10000, 6)

In [142]:
exchange_rate = np.reshape(exchange_rate, (10000,1))

In [143]:
numeric_data = np.hstack((numeric_data, exchange_rate)) #**how to use hstack(())

In [144]:
header_numeric.shape

(6,)

In [145]:
header_numeric = np.concatenate((header_numeric, np.array(['exchange_rate'])))
header_numeric

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt', 'exchange_rate'],
      dtype='<U19')

In [146]:
##header_numeric = np.delete(header_numeric,[-2,-1])

### From USD to EUR

In [147]:
header_numeric

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt', 'exchange_rate'],
      dtype='<U19')

In [148]:
columns_dollar = [1,2,4,5]

In [149]:
numeric_data[:,columns_dollar]

array([[35000.  , 35000.  ,  1184.86,  9452.96],
       [30000.  , 30000.  ,   938.57,  4679.7 ],
       [15000.  , 15000.  ,   494.86,  1969.83],
       ...,
       [10000.  , 10000.  ,  1372.97,  2185.64],
       [35000.  , 10000.  ,   354.3 ,  3199.4 ],
       [10000.  , 10000.  ,   309.97,   301.9 ]])

In [150]:
#exchange rate column is number 6
numeric_data[:,6]

array([1.1 , 1.1 , 1.13, ..., 1.12, 1.11, 1.09])

In [151]:
for i in columns_dollar: 
    numeric_data = np.hstack((numeric_data, np.reshape(numeric_data[:,i]/numeric_data[:,6],(10000,1)) ))

In [152]:
numeric_data

array([[48010226.  ,    35000.  ,    35000.  , ...,    31933.3 ,     1081.04,     8624.69],
       [57693261.  ,    30000.  ,    30000.  , ...,    27272.73,      853.25,     4254.27],
       [59432726.  ,    15000.  ,    15000.  , ...,    13326.3 ,      439.64,     1750.04],
       ...,
       [50415990.  ,    10000.  ,    10000.  , ...,     8910.3 ,     1223.36,     1947.47],
       [46154151.  ,    35000.  ,    10000.  , ...,     8997.4 ,      318.78,     2878.63],
       [66055249.  ,    10000.  ,    10000.  , ...,     9145.8 ,      283.49,      276.11]])

In [153]:
numeric_data.shape

(10000, 11)

### Expanding the header

In [154]:
header_additional = [column_name+'_EUR' for column_name in header_numeric[columns_dollar]]

In [155]:
np.array(header_additional)

array(['loan_amnt_EUR', 'funded_amnt_EUR', 'installment_EUR', 'total_pymnt_EUR'], dtype='<U15')

In [156]:
header_numeric = np.concatenate((header_numeric, header_additional)) #np.concatenate(())

In [157]:
header_numeric

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt', 'exchange_rate',
       'loan_amnt_EUR', 'funded_amnt_EUR', 'installment_EUR', 'total_pymnt_EUR'], dtype='<U19')

In [158]:
header_numeric[columns_dollar] = [column_name+'_USD' for column_name in header_numeric[columns_dollar]]

In [159]:
header_numeric

array(['id', 'loan_amnt_USD', 'funded_amnt_USD', 'int_rate', 'installment_USD', 'total_pymnt_USD',
       'exchange_rate', 'loan_amnt_EUR', 'funded_amnt_EUR', 'installment_EUR', 'total_pymnt_EUR'],
      dtype='<U19')

In [160]:
columns_index_order = [0,1,7,2,8,3,4,9,5,10,6]

In [161]:
header_numeric[columns_index_order]

array(['id', 'loan_amnt_USD', 'loan_amnt_EUR', 'funded_amnt_USD', 'funded_amnt_EUR', 'int_rate',
       'installment_USD', 'installment_EUR', 'total_pymnt_USD', 'total_pymnt_EUR', 'exchange_rate'],
      dtype='<U19')

In [163]:
header_numeric = header_numeric[columns_index_order]
header_numeric

array(['id', 'loan_amnt_USD', 'loan_amnt_EUR', 'funded_amnt_USD', 'funded_amnt_EUR', 'int_rate',
       'installment_USD', 'installment_EUR', 'total_pymnt_USD', 'total_pymnt_EUR', 'exchange_rate'],
      dtype='<U19')

In [168]:
numeric_data = numeric_data[:,columns_index_order]
numeric_data

In [171]:
header_numeric

array(['id', 'loan_amnt_USD', 'loan_amnt_EUR', 'funded_amnt_USD', 'funded_amnt_EUR', 'int_rate',
       'installment_USD', 'installment_EUR', 'total_pymnt_USD', 'total_pymnt_EUR', 'exchange_rate'],
      dtype='<U19')

In [172]:
numeric_data[:,5]
#we will need to convert this to decimal point

array([13.33, 28.99, 28.99, ..., 28.99, 16.55, 28.99])

In [173]:
numeric_data[:,5] = numeric_data[:,5]/100
numeric_data[:,5]

array([0.13, 0.29, 0.29, ..., 0.29, 0.17, 0.29])

### Checkpoint 2

In [174]:
checkpoint_numeric = checkpoint('Checkpoint_Numeric', header_numeric, numeric_data)

In [181]:
checkpoint_numeric["header"], checkpoint_numeric['data']

(array(['id', 'loan_amnt_USD', 'loan_amnt_EUR', 'funded_amnt_USD', 'funded_amnt_EUR', 'int_rate',
        'installment_USD', 'installment_EUR', 'total_pymnt_USD', 'total_pymnt_EUR', 'exchange_rate'],
       dtype='<U19'),
 array([[48010226.  ,    35000.  ,    31933.3 , ...,     9452.96,     8624.69,        1.1 ],
        [57693261.  ,    30000.  ,    27272.73, ...,     4679.7 ,     4254.27,        1.1 ],
        [59432726.  ,    15000.  ,    13326.3 , ...,     1969.83,     1750.04,        1.13],
        ...,
        [50415990.  ,    10000.  ,     8910.3 , ...,     2185.64,     1947.47,        1.12],
        [46154151.  ,    35000.  ,    31490.9 , ...,     3199.4 ,     2878.63,        1.11],
        [66055249.  ,    10000.  ,     9145.8 , ...,      301.9 ,      276.11,        1.09]]))

## Create the "Complete" Dataset

In [182]:
checkpoint_numeric['data'].shape

(10000, 11)

In [183]:
checkpoint_strings['data'].shape

(10000, 6)

In [186]:
np.hstack((checkpoint_numeric['data'],checkpoint_strings['data']))

array([[48010226.  ,    35000.  ,    31933.3 , ...,       13.  ,        1.  ,        1.  ],
       [57693261.  ,    30000.  ,    27272.73, ...,        5.  ,        1.  ,        4.  ],
       [59432726.  ,    15000.  ,    13326.3 , ...,       10.  ,        1.  ,        4.  ],
       ...,
       [50415990.  ,    10000.  ,     8910.3 , ...,        5.  ,        1.  ,        1.  ],
       [46154151.  ,    35000.  ,    31490.9 , ...,       17.  ,        1.  ,        3.  ],
       [66055249.  ,    10000.  ,     9145.8 , ...,        4.  ,        0.  ,        3.  ]])

In [187]:
np.hstack((checkpoint_numeric['data'],checkpoint_strings['data'])).shape

(10000, 17)

In [188]:
loan_data = np.hstack((checkpoint_numeric['data'],checkpoint_strings['data']))

In [189]:
loan_data

array([[48010226.  ,    35000.  ,    31933.3 , ...,       13.  ,        1.  ,        1.  ],
       [57693261.  ,    30000.  ,    27272.73, ...,        5.  ,        1.  ,        4.  ],
       [59432726.  ,    15000.  ,    13326.3 , ...,       10.  ,        1.  ,        4.  ],
       ...,
       [50415990.  ,    10000.  ,     8910.3 , ...,        5.  ,        1.  ,        1.  ],
       [46154151.  ,    35000.  ,    31490.9 , ...,       17.  ,        1.  ,        3.  ],
       [66055249.  ,    10000.  ,     9145.8 , ...,        4.  ,        0.  ,        3.  ]])

In [191]:
np.isnan(loan_data).sum()

0

In [193]:
header_full = np.concatenate((header_numeric,header_strings))

In [194]:
header_full

array(['id', 'loan_amnt_USD', 'loan_amnt_EUR', 'funded_amnt_USD', 'funded_amnt_EUR', 'int_rate',
       'installment_USD', 'installment_EUR', 'total_pymnt_USD', 'total_pymnt_EUR', 'exchange_rate',
       'issue_data', 'loan_status', 'term_month', 'sub_grade', 'verification_status', 'addr_state'],
      dtype='<U19')

## Sorting the New Dataset

In [197]:
np.sort(loan_data[:,0])

array([  373332.,   575239.,   707689., ..., 68614880., 68615915., 68616519.])

In [198]:
np.argsort(loan_data[:,0])

array([2086, 4812, 2353, ..., 4935, 9388, 8415], dtype=int64)

In [199]:
 loan_data[np.argsort(loan_data[:,0])]

array([[  373332.  ,     9950.  ,     9038.08, ...,       21.  ,        0.  ,        1.  ],
       [  575239.  ,    12000.  ,    10900.2 , ...,       25.  ,        1.  ,        2.  ],
       [  707689.  ,    10000.  ,     8924.3 , ...,       13.  ,        1.  ,        0.  ],
       ...,
       [68614880.  ,     5600.  ,     5121.65, ...,        8.  ,        1.  ,        1.  ],
       [68615915.  ,     4000.  ,     3658.32, ...,       10.  ,        1.  ,        2.  ],
       [68616519.  ,    21600.  ,    19754.93, ...,        3.  ,        0.  ,        2.  ]])

In [200]:
loan_data =  loan_data[np.argsort(loan_data[:,0])]
loan_data

array([[  373332.  ,     9950.  ,     9038.08, ...,       21.  ,        0.  ,        1.  ],
       [  575239.  ,    12000.  ,    10900.2 , ...,       25.  ,        1.  ,        2.  ],
       [  707689.  ,    10000.  ,     8924.3 , ...,       13.  ,        1.  ,        0.  ],
       ...,
       [68614880.  ,     5600.  ,     5121.65, ...,        8.  ,        1.  ,        1.  ],
       [68615915.  ,     4000.  ,     3658.32, ...,       10.  ,        1.  ,        2.  ],
       [68616519.  ,    21600.  ,    19754.93, ...,        3.  ,        0.  ,        2.  ]])

In [202]:
np.argsort(loan_data[:,0])

array([   0,    1,    2, ..., 9997, 9998, 9999], dtype=int64)

In [203]:
np.argmax(loan_data[:,0])

9999

## Store the New Dataset

In [204]:
np.vstack((header_full,loan_data))

array([['id', 'loan_amnt_USD', 'loan_amnt_EUR', ..., 'sub_grade', 'verification_status',
        'addr_state'],
       ['373332.0', '9950.0', '9038.082814338286', ..., '21.0', '0.0', '1.0'],
       ['575239.0', '12000.0', '10900.20037910145', ..., '25.0', '1.0', '2.0'],
       ...,
       ['68614880.0', '5600.0', '5121.647851612413', ..., '8.0', '1.0', '1.0'],
       ['68615915.0', '4000.0', '3658.319894008867', ..., '10.0', '1.0', '2.0'],
       ['68616519.0', '21600.0', '19754.927427647883', ..., '3.0', '0.0', '2.0']], dtype='<U32')

In [205]:
loan_data = np.vstack((header_full,loan_data))
loan_data

array([['id', 'loan_amnt_USD', 'loan_amnt_EUR', ..., 'sub_grade', 'verification_status',
        'addr_state'],
       ['373332.0', '9950.0', '9038.082814338286', ..., '21.0', '0.0', '1.0'],
       ['575239.0', '12000.0', '10900.20037910145', ..., '25.0', '1.0', '2.0'],
       ...,
       ['68614880.0', '5600.0', '5121.647851612413', ..., '8.0', '1.0', '1.0'],
       ['68615915.0', '4000.0', '3658.319894008867', ..., '10.0', '1.0', '2.0'],
       ['68616519.0', '21600.0', '19754.927427647883', ..., '3.0', '0.0', '2.0']], dtype='<U32')

In [206]:
np.savetxt('loan-data-preprocessed.csv',
            loan_data,
            fmt = '%s', #format as str
            delimiter=',')

In [207]:
#the data has been cleaned, preprocessed, and saved!